# Количество продаж, процент плохих отзывов, доходность транзакций

Компания, занимающаяся оптовыми поставками зернового кофе, запросила отчёт о продажах по месяцам. Интересуют только те продажи, у которых не было доставки: их количество в месяц, процент плохих отзывов, доходность таких транзакций – в разрезе на продажи с вводом клиентом  промокода и без. 
Имеется выгрузка всех продаж с момента основания компании.
Необходимо написать код на Python, который прочтет выгрузку, проведет необходимые расчеты и запишет таблицу-результат в эксель. 
Поля итоговой таблицы: 
«Месяц» – «Введен промокод» – «Кол-во транзакций» – «Плохих отзывов, %» - «Доходность, %».

Примечание:
Доходность можно вычислить по формуле
Доходность, % = (Оплаченная по транзакциям сумма - Закупочная цена поставленных зерен) / Закупочная цена поставленных зерен * 100, где
Закупочная цена поставленных зерен = сумма Cost всех с нужными характеристиками,
Оплаченная по транзакциям сумма= сумма Paid транзакций с нужными характеристиками.


In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [9]:
data=pd.read_csv('Транзакции.csv', encoding='cp1251', sep='\t')
data

,TransactionID,Date,Count,Cost,Доставка,Плохой отзыв,Paid,Weight,Promo,AutoIssue
0,1,2020-Aug-21 8:53:03,67,6325,0,0,"2605,0",5481,0,0
1,2,2019-Sep-3 11:53:03,90,8307,1,1,"8553,0",5956,1,1
2,3,2019-Oct-15 4:31:02,59,9819,0,1,"9995,0",5170,1,0
3,4,2019-Aug-26 4:14:00,14,5351,1,1,"12603,0",8826,0,0
4,5,2020-Aug-27 1:21:03,14,7779,1,0,"5625,0",5357,0,0
...,...,...,...,...,...,...,...,...,...,...
14360,14361,2019-Oct-30 4:42:01,70,4716,1,1,"3201,0",7509,1,1
14361,14362,2019-Oct-8 23:23:03,14,4547,1,0,"10810,0",8511,1,0
14362,14363,2020-Aug-17 18:25:00,8,9940,1,1,"10597,0",9649,0,1
14363,14364,2020-Aug-27 9:38:01,92,7180,0,0,"7200,0",6638,0,0


In [10]:
data=data.set_index('TransactionID')
data

,Date,Count,Cost,Доставка,Плохой отзыв,Paid,Weight,Promo,AutoIssue
TransactionID,,,,,,,,,
1,2020-Aug-21 8:53:03,67,6325,0,0,"2605,0",5481,0,0
2,2019-Sep-3 11:53:03,90,8307,1,1,"8553,0",5956,1,1
3,2019-Oct-15 4:31:02,59,9819,0,1,"9995,0",5170,1,0
4,2019-Aug-26 4:14:00,14,5351,1,1,"12603,0",8826,0,0
5,2020-Aug-27 1:21:03,14,7779,1,0,"5625,0",5357,0,0
...,...,...,...,...,...,...,...,...,...
14361,2019-Oct-30 4:42:01,70,4716,1,1,"3201,0",7509,1,1
14362,2019-Oct-8 23:23:03,14,4547,1,0,"10810,0",8511,1,0
14363,2020-Aug-17 18:25:00,8,9940,1,1,"10597,0",9649,0,1


In [11]:
data1 = data[data.Доставка == 0]
data1

,Date,Count,Cost,Доставка,Плохой отзыв,Paid,Weight,Promo,AutoIssue
TransactionID,,,,,,,,,
1,2020-Aug-21 8:53:03,67,6325,0,0,"2605,0",5481,0,0
3,2019-Oct-15 4:31:02,59,9819,0,1,"9995,0",5170,1,0
8,2020-Aug-23 13:34:03,45,6381,0,0,"6251,0",9227,0,0
9,2020-Aug-31 16:57:02,31,8795,0,1,"2047,0",6044,0,1
10,2020-Aug-29 9:18:01,41,4175,0,1,"6300,0",7866,1,1
...,...,...,...,...,...,...,...,...,...
14355,2019-Dec-2 15:52:01,36,6704,0,0,"8613,0",7665,0,0
14357,2020-Aug-26 8:30:05,68,6042,0,1,"10181,0",5656,1,1
14359,2020-Aug-27 15:22:02,67,5211,0,0,"3873,0",4345,1,1


In [12]:
data1.Date = data1.Date.map(lambda x: x.split(' ')[0].split('-')[:2])
data1.Date

C:\Users\STerenteva\AppData\Local\Temp\ipykernel_14696\2885769535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.Date = data1.Date.map(lambda x: x.split(' ')[0].split('-')[:2])


TransactionID
1        [2020, Aug]
3        [2019, Oct]
8        [2020, Aug]
9        [2020, Aug]
10       [2020, Aug]
            ...     
14355    [2019, Dec]
14357    [2020, Aug]
14359    [2020, Aug]
14360    [2020, Aug]
14364    [2020, Aug]
Name: Date, Length: 7128, dtype: object

In [13]:
data1.Date = data1.Date.map(lambda y: '-'.join(y))

C:\Users\STerenteva\AppData\Local\Temp\ipykernel_14696\1637943371.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.Date = data1.Date.map(lambda y: '-'.join(y))


In [14]:
data1

,Date,Count,Cost,Доставка,Плохой отзыв,Paid,Weight,Promo,AutoIssue
TransactionID,,,,,,,,,
1,2020-Aug,67,6325,0,0,"2605,0",5481,0,0
3,2019-Oct,59,9819,0,1,"9995,0",5170,1,0
8,2020-Aug,45,6381,0,0,"6251,0",9227,0,0
9,2020-Aug,31,8795,0,1,"2047,0",6044,0,1
10,2020-Aug,41,4175,0,1,"6300,0",7866,1,1
...,...,...,...,...,...,...,...,...,...
14355,2019-Dec,36,6704,0,0,"8613,0",7665,0,0
14357,2020-Aug,68,6042,0,1,"10181,0",5656,1,1
14359,2020-Aug,67,5211,0,0,"3873,0",4345,1,1


In [15]:
result1 = data1.groupby(['Date','Promo']).Count.sum().reset_index()
result1.columns = ['Месяц', 'Введен промокод', 'Кол-во транзакций']
result1

,Месяц,Введен промокод,Кол-во транзакций
0,2019-Apr,0,8932
1,2019-Apr,1,7115
2,2019-Aug,0,9756
3,2019-Aug,1,7508
4,2019-Dec,0,11602
5,2019-Dec,1,10850
6,2019-Jul,0,8622
7,2019-Jul,1,10198
8,2019-Jun,0,9197
9,2019-Jun,1,9239


In [22]:
sum_bad = data1.groupby(['Date','Promo'])["Плохой отзыв"].sum()
sum_bad

Date      Promo
2019-Apr  0         88
          1         64
2019-Aug  0         82
          1         84
2019-Dec  0        113
          1        119
2019-Jul  0         68
          1        103
2019-Jun  0         88
          1         85
2019-Mar  0          1
2019-May  0        100
          1         82
2019-Nov  0        126
          1        119
2019-Oct  0        129
          1        135
2019-Sep  0        113
          1         91
2020-Apr  0         71
          1         98
2020-Aug  0        107
          1        138
2020-Feb  0        116
          1         82
2020-Jan  0        107
          1        108
2020-Jul  0        129
          1        119
2020-Jun  0         92
          1        101
2020-Mar  0        151
          1        149
2020-May  0         99
          1        105
Name: Плохой отзыв, dtype: int64

In [18]:
sum_all = data1.groupby(['Date','Promo'])["Плохой отзыв"].count()
sum_all

Date      Promo
2019-Apr  0        177
          1        139
2019-Aug  0        183
          1        147
2019-Dec  0        225
          1        212
2019-Jul  0        159
          1        186
2019-Jun  0        181
          1        181
2019-Mar  0          1
2019-May  0        187
          1        179
2019-Nov  0        235
          1        237
2019-Oct  0        238
          1        272
2019-Sep  0        225
          1        200
2020-Apr  0        167
          1        188
2020-Aug  0        245
          1        255
2020-Feb  0        218
          1        189
2020-Jan  0        216
          1        220
2020-Jul  0        241
          1        232
2020-Jun  0        195
          1        198
2020-Mar  0        295
          1        294
2020-May  0        201
          1        210
Name: Плохой отзыв, dtype: int64

In [19]:
result2 = round(sum_bad/sum_all*100, 2)
result2 = result2.reset_index()['Плохой отзыв']
result2

0      49.72
1      46.04
2      44.81
3      57.14
4      50.22
5      56.13
6      42.77
7      55.38
8      48.62
9      46.96
10    100.00
11     53.48
12     45.81
13     53.62
14     50.21
15     54.20
16     49.63
17     50.22
18     45.50
19     42.51
20     52.13
21     43.67
22     54.12
23     53.21
24     43.39
25     49.54
26     49.09
27     53.53
28     51.29
29     47.18
30     51.01
31     51.19
32     50.68
33     49.25
34     50.00
Name: Плохой отзыв, dtype: float64

In [20]:
result1['Плохих отзывов, %'] = result2
result1

,Месяц,Введен промокод,Кол-во транзакций,"Плохих отзывов, %"
0,2019-Apr,0,8932,49.72
1,2019-Apr,1,7115,46.04
2,2019-Aug,0,9756,44.81
3,2019-Aug,1,7508,57.14
4,2019-Dec,0,11602,50.22
5,2019-Dec,1,10850,56.13
6,2019-Jul,0,8622,42.77
7,2019-Jul,1,10198,55.38
8,2019-Jun,0,9197,48.62
9,2019-Jun,1,9239,46.96


In [21]:
result1.to_excel('result.xlsx')

# Доходность

Доходность можно вычислить по формуле Доходность, % = (Оплаченная по транзакциям сумма - Закупочная цена поставленных зерен) / Закупочная цена поставленных зерен * 100, где Закупочная цена поставленных зерен = сумма Cost всех с нужными характеристиками, Оплаченная по транзакциям сумма= сумма Paid транзакций с нужными характеристиками.

In [47]:
data1

,Date,Count,Cost,Доставка,Плохой отзыв,Paid,Weight,Promo,AutoIssue
TransactionID,,,,,,,,,
1,2020-Aug,67,6325,0,0,2605,5481,0,0
3,2019-Oct,59,9819,0,1,9995,5170,1,0
8,2020-Aug,45,6381,0,0,6251,9227,0,0
9,2020-Aug,31,8795,0,1,2047,6044,0,1
10,2020-Aug,41,4175,0,1,6300,7866,1,1
...,...,...,...,...,...,...,...,...,...
14355,2019-Dec,36,6704,0,0,8613,7665,0,0
14357,2020-Aug,68,6042,0,1,10181,5656,1,1
14359,2020-Aug,67,5211,0,0,3873,4345,1,1


In [49]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7128 entries, 1 to 14364
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          7128 non-null   object
 1   Count         7128 non-null   int64 
 2   Cost          7128 non-null   int64 
 3   Доставка      7128 non-null   int64 
 4   Плохой отзыв  7128 non-null   int64 
 5   Paid          7128 non-null   int64 
 6   Weight        7128 non-null   int64 
 7   Promo         7128 non-null   int64 
 8   AutoIssue     7128 non-null   int64 
dtypes: int64(8), object(1)
memory usage: 556.9+ KB


Поле Paid имеет формат object. Требуется изменить на int64, чтобы производить вычисления.

In [46]:
data1.Paid.map(lambda x: x.split(',')[0]).astype('int64')

AttributeError: 'int' object has no attribute 'split'

In [55]:
result1['Доходность, %'] = (data1.Cost - data1.Paid)/data1.Paid*100
result1

,Месяц,Введен промокод,Кол-во транзакций,"Плохих отзывов, %","Доходность, %"
0,2019-Apr,0,8932,49.72,NaN
1,2019-Apr,1,7115,46.04,142.802303
2,2019-Aug,0,9756,44.81,NaN
3,2019-Aug,1,7508,57.14,-1.760880
4,2019-Dec,0,11602,50.22,NaN
5,2019-Dec,1,10850,56.13,NaN
6,2019-Jul,0,8622,42.77,NaN
7,2019-Jul,1,10198,55.38,NaN
8,2019-Jun,0,9197,48.62,2.079667
9,2019-Jun,1,9239,46.96,329.653151
